# Final Model

## Libraries

In [2]:
# main
import pandas as pd
import numpy as np
import time

# spotify
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

## Setting everything for running the model

### Spotify API

In [3]:
credentials = []

with open("spotify_credentials.txt", "r") as spotify:
    for line in spotify:
        line = line.rstrip("\n")
        credentials.append(line)
        
#Initialize SpotiPy with user credentias
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(
    client_id = credentials[0],
    client_secret = credentials[1]))

### Scaler

In [4]:
spotify_playlist = pd.read_csv("../data/spotify_playlist.csv")
final_list = spotify_playlist.iloc[:, 2:]

In [5]:
# initilazing the scaler
scaler = StandardScaler()

# scaling the playlist
playlist_scaled = scaler.fit_transform(final_list)

### KMeans

In [6]:
# intilize the model
kmeans = KMeans(n_clusters=7, random_state=42)

# fit the model
kmeans.fit(playlist_scaled)

KMeans(n_clusters=7, random_state=42)

## Running the Final Model

### Loading the CSV Files

In [7]:
billboard = pd.read_csv("../data/billboard100.csv")
playlist = pd.read_csv("../data/playlist_with_clusters.csv")

### Testing the Model

In [8]:
#User inputs a song. If it's on the DF, it will return another one, if not it will search on Spotify.
def input_song(df):
    choice = input("Welcome to the Song Recommender Pro-9000. Tell me one song and I will recommend you another one:\n").lower()
    
    if choice in df.str.lower().values:
        return 1, choice
    else:
        return 0, choice

In [9]:
song, choice = input_song(billboard["song"])

if song == 1:
    suggestion = billboard.sample()
    suggestion_song = suggestion["song"].values.flatten()
    suggestion_artist = suggestion["artist(s)"].values.flatten()
    suggestion_song = str(suggestion_song).lstrip("[").rstrip("]")
    suggestion_artist = str(suggestion_artist).lstrip("[").rstrip("]")

    time.sleep(1.5)
    print("\nLoading..")
    time.sleep(1.5)
    print("Loading...\n")
    time.sleep(0.5)
    print("We found something!\n")
    print(f"Here's our recommedation: {suggestion_song} from {suggestion_artist}. What a hit!")

elif song == 0:
    spoti_song = spotify.search(q=choice)
        
    print(spoti_song["tracks"]["items"][0]["name"], " - ", spoti_song["tracks"]["items"][0]["artists"][0]["name"])
    song_features_spoti = pd.DataFrame(spotify.audio_features(spoti_song["tracks"]["items"][0]["uri"]))
    spoti_features = song_features_spoti[["danceability", "energy", "mode", "acousticness", "instrumentalness", "liveness",
                          "loudness", "speechiness", "valence", "tempo"]]

    features_spoti_scaled = scaler.transform(spoti_features)
    spoti_cluster = kmeans.predict(features_spoti_scaled)
    match_song = playlist[playlist["cluster"] == int(spoti_cluster)]["song"]
    
    song_suggestion = match_song.sample()

    clean_song = list(song_suggestion)
    clean_song_2 = str(clean_song).lstrip("[").rstrip("]")    

    print("Wops, seems your song is not hot right now! Let me find you something else.\n")

    time.sleep(1.5)
    print("Loading..")
    time.sleep(1.5)
    print("Loading...\n")
    time.sleep(0.5)

    print("Finally! Here's your perfect song:\n")
    print(f"{clean_song_2}\n")


Welcome to the Song Recommender Pro-9000. Tell me one song and I will recommend you another one:
 Danza Kuduro


Danza Kuduro  -  Don Omar
Wops, seems your song is not hot right now! Let me find you something else.

Loading..
Loading...

Finally! Here's your perfect song:

'All Day'



Do you want to predict another song? y/n:  y


Here's another recommendation:



NameError: name 'match_table_spoti_input' is not defined